<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2023notebooks/2023_0614karpathy_Policy_gradient_pong.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Reinforcement Learning: Pong from Pixels

## 方策勾配法 (Policy gradient) を用いた強化学習エージェントの学習 (OpenAI gym)

- [Original `pg-pong.py`](https://gist.github.com/karpathy/a4166c7fe253700972fcbc77e4ea32c5)
- このファイルは Andrej Karpathy の [ブログ記事](http://karpathy.github.io/2016/05/31/rl/) にあった `pg-pong.py` を Python3 で動くように修正し，かつ colaboratory で実行できるよう修正したものです。優れたブログ記事を書いた Karpathy に敬意を評します。

---

<div>
    <center>
        <img src="http://karpathy.github.io/assets/rl/preview.jpeg" style="width:69%"><br>
        <strong>市井の強化学習。左からアタリのゲームをするディープ Q ネット，アルファ碁，バークレイロボットによるレゴ積み木，四足歩行の物理シミュレーション Karpathy のブログより</strong>
    </center>
</div>

ちなみに Karpathy はこのブログの中で，最近の進歩は次の 4 点が大きいと書いています。

1. 計算  (the obvious one: Moore’s Law, GPUs, ASICs),
1. データ (in a nice form, not just out there somewhere on the internet - e.g. ImageNet),
1. アルゴリズム (research and ideas, e.g. backprop, CNN, LSTM), and
1. インフラ (software under you - Linux, TCP/IP, Git, ROS, PR2, AWS, AMT, TensorFlow, etc.).
---

<div>
    <center>
        <img src="http://karpathy.github.io/assets/rl/policy.png" style="width:39%"><br>
        <strong>2層のポリシーネットワーク</strong>
    </center>
</div>

---

次の2つの図は通常の教師ありニューラルネットワークと強化学習との違いを表しています。
<div>
    <center>
        <img src="http://karpathy.github.io/assets/rl/sl.png" style="width:74%"><br>
        <strong>通常のニューラルネットワーク</strong>
    </center>
</div>

<div>
    <center>
        <img src="http://karpathy.github.io/assets/rl/rl.png" style="width:74%"><br>
        <strong>強化学習</strong>
    </center>
</div>

\begin{align}
\nabla_{\theta} \mathbb{E}_x\left[f(x)\right] &= \nabla_{\theta} \sum_x p(x) f(x) & \text{期待値の定義 definition of expectation} \\
& = \sum_x \nabla_{\theta} p(x) f(x) & \text{総和と勾配の入れ替え swap sum and gradient} \\
& = \sum_x p(x) \frac{\nabla_{\theta} p(x)}{p(x)} f(x) & \text{掛けて割る both multiply and divide by } p(x) \\
& = \sum_x p(x) \nabla_{\theta} \log p(x) f(x) & \text{次の公式を用いる use the fact that } \nabla_{\theta} \log(z) = \frac{1}{z} \nabla_{\theta} z \\
& = \mathbb{E}_x\left[f(x) \nabla_{\theta} \log p(x) \right] & \text{definition of expectation}
\end{align}


In [ ]:
#何が起こっているかを知りたい場合には下のコマンド中不等号以下をremove " > /dev/null 2>&1" を削除してください
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
#!pip install ez_setup > /dev/null 2>&1

import urllib.request
urllib.request.urlretrieve('http://www.atarimania.com/roms/Roms.rar','Roms.rar')
!pip install unrar
!unrar x Roms.rar
!mkdir rars
!mv HC\ ROMS.zip   rars
!mv ROMS.zip  rars
!python -m atari_py.import_roms rars


# 2022_1122
# gym のバージョンの違いが影響するらしい
#!pip install gym==0.25.2
# 2023_0614 更新，最新バージョンにしてみる
!pip install 'gym[atari, accept-rom-license]'
#!pip install 'gym[atari, accept-rom-license]'==0.22.0
!pip install ale-py==0.7.4

""" Trains an agent with (stochastic) Policy Gradients on Pong. Uses OpenAI Gym. """
from __future__ import print_function
from six.moves import xrange
from six.moves import cPickle

import numpy as np
#import cPickle as pickle
import pickle
import gym

In [2]:
""" Trains an agent with (stochastic) Policy Gradients on Pong. Uses OpenAI Gym. """
from __future__ import print_function
from six.moves import xrange
from six.moves import cPickle

import numpy as np
#import cPickle as pickle
import pickle
import gym

In [3]:
# hyperparameters
H = 200 # 中間層のニューロン数
batch_size = 10 # パラメータ更新を行うエピソード回数 every how many episodes to do a param update?
learning_rate = 1e-4 # 学習率
gamma = 0.99 # 報酬の割引率 discount factor for reward
decay_rate = 0.99 # 勾配の 2 乗 に対する RMSProp の崩壊率 decay factor for RMSProp leaky sum of grad^2
resume = False # 以前学習した情報を用いるか否か resume from previous checkpoint?
render = False

In [4]:
# model initialization
D = 80 * 80 # input dimensionality: 80x80 grid
if resume:
    model = pickle.load(open('save.p', 'rb'))
else:
    model = {}
    model['W1'] = np.random.randn(H,D) / np.sqrt(D) # ザビエルの初期化 "Xavier" initialization
    model['W2'] = np.random.randn(H) / np.sqrt(H)

#update buffers that add up gradients over a batch
#grad_buffer = { k : np.zeros_like(v) for k,v in model.iteritems() }
grad_buffer = {k:np.zeros_like(v) for k,v in model.items()}     # python 3 対応のため書き換え

#RMSProm 用の記憶バッファ rmsprop memory
#rmsprop_cache = { k : np.zeros_like(v) for k,v in model.iteritems() }
rmsprop_cache = {k:np.zeros_like(v) for k,v in model.items()}

def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

def prepro(I):
    """ prepro 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """
    I = I[35:195] # crop
    I = I[::2,::2,0] # downsample by factor of 2
    I[I == 144] = 0 # erase background (background type 1)
    I[I == 109] = 0 # erase background (background type 2)
    I[I != 0] = 1 # everything else (paddles, ball) just set to 1
    return I.astype(float).ravel()
    #return I.astype(np.float).ravel()

def discount_rewards(r):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(xrange(0, r.size)):
        if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

def policy_forward(x):
    h = np.dot(model['W1'], x)
    h[h<0] = 0 # ReLU nonlinearity
    logp = np.dot(model['W2'], h)
    p = sigmoid(logp)
    return p, h # return probability of taking action 2, and hidden state

def policy_backward(eph, epdlogp):
    """ backward pass. (eph is array of intermediate hidden states) """
    dW2 = np.dot(eph.T, epdlogp).ravel()
    dh = np.outer(epdlogp, model['W2'])
    dh[eph <= 0] = 0 # backpro prelu
    dW1 = np.dot(dh.T, epx)
    return {'W1':dW1, 'W2':dW2}

In [5]:
#env = gym.make("Pong-v4")
env = gym.make("Pong-v0")
observation = env.reset()
prev_x = None # used in computing the difference frame
xs,hs,dlogps,drs = [],[],[],[]
running_reward = None
reward_sum = 0
episode_number = 0

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment Pong-v0 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/utils/seeding.py:159: DeprecationWarning: WARN: Function `hash_seed(seed, max_bytes)` is marked as deprecated and will be removed in the future. 
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/seeding.py:203: DeprecationWarning: WARN: Function `_bigint_from_bytes(bytes)` is marked as deprecated and will be removed in the future. 
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initi

---

<font size="+2" color="red">注:以下を実行すると無限ループです。適当なところで止めてください</font>


In [6]:
render = True

In [ ]:
while True:
    if render:
        env.render(mode='ansi')
        #env.render(mode='rgb_array')
        #env.render(mode='human') # error
        #env.render()

    # preprocess the observation, set input to network to be difference image
    cur_x = prepro(observation)
    x = cur_x - prev_x if prev_x is not None else np.zeros(D)
    prev_x = cur_x

    # forward the policy network and sample an action from the returned probability
    aprob, h = policy_forward(x)
    action = 2 if np.random.uniform() < aprob else 3 # roll the dice!

    # record various intermediates (needed later for backprop)
    xs.append(x) # observation
    hs.append(h) # hidden state
    y = 1 if action == 2 else 0 # a "fake label"
    dlogps.append(y - aprob)
    # grad that encourages the action that was taken to be taken
    #(see http://cs231n.github.io/neural-networks-2/#losses if confused)

    # step the environment and get new measurements
    observation, reward, done, info = env.step(action)
    reward_sum += reward

    drs.append(reward) # record reward (has to be done after we call step()
                       #  to get reward for previous action)

    if done: # an episode finished
        episode_number += 1

        # stack together all inputs, hidden states, action gradients, and rewards for this episode
        epx = np.vstack(xs)
        eph = np.vstack(hs)
        epdlogp = np.vstack(dlogps)
        epr = np.vstack(drs)
        xs,hs,dlogps,drs = [],[],[],[] # reset array memory

        # compute the discounted reward backwards through time
        discounted_epr = discount_rewards(epr)
        # standardize the rewards to be unit normal (helps control the gradient estimator variance)
        discounted_epr -= np.mean(discounted_epr)
        discounted_epr /= np.std(discounted_epr)

        epdlogp *= discounted_epr # modulate the gradient with advantage (PG magic happens right here.)
        grad = policy_backward(eph, epdlogp)
        for k in model: grad_buffer[k] += grad[k] # accumulate grad over batch

        # perform rmsprop parameter update every batch_size episodes
        if episode_number % batch_size == 0:
            for k,v in model.items():
                g = grad_buffer[k] # gradient
                rmsprop_cache[k] = decay_rate * rmsprop_cache[k] + (1 - decay_rate) * g**2
                model[k] += learning_rate * g / (np.sqrt(rmsprop_cache[k]) + 1e-5)
                grad_buffer[k] = np.zeros_like(v) # reset batch gradient buffer

        # boring book-keeping
        running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
        print('resetting env. episode reward total was %f. running mean: %f' % (reward_sum, running_reward))
        if episode_number % 100 == 0: pickle.dump(model, open('save.p', 'wb'))
        reward_sum = 0
        observation = env.reset() # reset env
        prev_x = None

    if reward != 0: # Pong has either +1 or -1 reward exactly when game ends.
        print('ep %d: game finished, reward: %f' % (episode_number, reward), end='')
        print('') if reward == -1 else ' !!!!!!!!'